# This notebook demonstrates the analysis carried on the **divvy bicycles** data set:

Dataset:
<br>
__[My Google Drive](https://drive.google.com/drive/folders/17onvjg5MslNtkLzfv--sIb4yAa4cNe3t?usp=sharing)__

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt


In [ ]:
# matplotlib.style.use('ggplot')
# sc= SparkContext(appName="my_app3", master= "local[1]") 
# sqlContext = SQLContext(sc) 